In [68]:
import torch
import torch.nn as nn
import numpy as np

$S(y_{i})=\frac{e^{y_{i}}}{\sum_{k}e^{y_{k}}}$

In [69]:
# Native

In [70]:
def softmax(y):
    return np.exp(y) / np.sum(np.exp(y), axis=0)  # np.exp(y) => e^y

In [71]:
y = np.array([2.0, 1.0, 0.1])
outputs = softmax(y)
print(f'Softmax numpy: {outputs}')

Softmax numpy: [0.65900114 0.24243297 0.09856589]


In [72]:
# Pytorch method

In [73]:
x = torch.tensor([2.0, 1.0, 0.1], dtype=torch.float32)
outputs = torch.softmax(x, dim=0)  # dim -> dimension
outputs

tensor([0.6590, 0.2424, 0.0986])

With normalization multiplier (boltzmann distribution; t = temperature)
$S(y_{i})=\frac{e^{y_{i}/t}}{\sum_{k}e^{y_{k}/t}}$

In [74]:
# Cross entropy (loss function) BCE

In [75]:
# Native
def cross_entropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss  # / float(predicted.shape[0])

In [76]:
# y - must be one hot encoded
# if class 0: [1, 0, 0]
# if class 1: [0, 1, 0]
# if class 2: [0, 0, 1]

Y = np.array([1, 0, 0])

In [77]:
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)

print(f'Loss1 numpy: {l1}')
print(f'Loss2 numpy: {l2}')

# The higher the value, the greater losses (the greater error)

Loss1 numpy: 0.35667494393873245
Loss2 numpy: 2.3025850929940455


In [78]:
# pytorch
# nn.CrossEntropyLoss() excludes:
# 1) Not use Softmax in last layer!
# 2) Not use One Hot!
# Because: nn.CrossEntropyLoss() is nn.LogSoftmax + nn.NLLLoss
loss = nn.CrossEntropyLoss()

In [79]:
Y = torch.tensor([0])

# n_samples x n_classes = 1 x 3
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'Loss 1: {l1.item()}')
print(f'Loss 2: {l2.item()}')

Loss 1: 0.4170299470424652
Loss 2: 1.840616226196289


In [80]:
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
display(predictions1, predictions2)

tensor([0])

tensor([1])

In [81]:
# several samples
Y = torch.tensor([2, 0, 1])

# n_samples x n_classes = 3 x 3

Y_pred_good = torch.tensor([[0.1, 1.0, 2.1], [2.0, 1.0, 0.1], [0.1, 3.0, 0.1]])
Y_pred_bad = torch.tensor([[2.1, 1.0, 0.1], [0.1, 1.0, 2.1], [0.1, 3.0, 0.1]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'Loss 1: {l1.item()}')
print(f'Loss 2: {l2.item()}')

Loss 1: 0.3018244206905365
Loss 2: 1.6241613626480103


In [82]:
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
display(predictions1, predictions2)

tensor([2, 0, 1])

tensor([0, 2, 1])

In [83]:
# Multiclass problem
# if value of classes > 2
# We ask a question: What comes out of => e.g select [car, bike, home, dog]

class NeuralNetMulti(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetMulti, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)

        # no softmax at the end
        return out

In [84]:
model = NeuralNetMulti(input_size=28*28, hidden_size=5, num_classes=2)

In [85]:
criterion = nn.CrossEntropyLoss()

In [86]:
# Binary classification
# if value of classes = 2
# We ask a question: It is dog? => select [yes, no]

class NeuralNetBinary(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNetBinary, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)

        y_pred = torch.sigmoid(out)

        return y_pred

In [87]:
model = NeuralNetBinary(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss()